In [45]:
import torch
import pandas as pd
from torch import nn
from torch.utils.data import Dataset
from collections import Counter
from torchtext.vocab import vocab
from tqdm import tqdm
import ast


class BertDataSet(Dataset) :
    def __init__(self,dir,index_col=0) -> None:
        super().__init__()
        self.data_load = pd.read_csv(dir,index_col=index_col)
        self.counter = Counter()
        self.vocab = None
        self.sentences = self._merge_sentences()

    def _change_string_to_list(self,str_list):
        """

        dataframe안에 list를 통으로 넣으면 str으로 저장된다.
        ast 라이브러리를 쓰면 원래 ㅣist 이지만 str 타입으로 표현된 값을 다시 list 타입으로 바꿔준다.

        """
        return ast.literal_eval(str_list)

    def _merge_sentences(self) :
        """
        
        하나의 list 안에 모든 sentence 넣기

        """
        total = []
        for row in self.data_load.iterrows() :
            book_info = row[1]
            t = []
            for i in range(1,4) :
                t += self._change_string_to_list(book_info.iloc[i])

            total += t
        total = list(filter(None,total))
        
        return total

    

    # def __getitem__() :
    #     pass
    # def __call__(self, ) :
    #     pass


In [7]:
# 리스트 하나에 문장 전체 넣기
# counter로 단어 중복 제거하기
# vocab으로 index와 단어 매치하기 

In [46]:
test = BertDataSet('./data/bookraw_total.csv')
pd.DataFrame(test.sentences)[0].to_csv('./data/bookraw_list.txt',index=False)

### Wordpiece Tokenizer 학습하기

In [64]:
import sentencepiece as spm
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(lowercase=True, strip_accents=False)

tokenizer.train('./data/bookraw_list.txt',vocab_size=100000,limit_alphabet=6000, min_frequency=10)

# Tokenizing Test
tokenizer.encode('python과 javascript를 만들며 고민했다').tokens

# Saving Vocab
tokenizer.save_model('.', 'bert')

### Tokenizing 한 것 불러오기

In [1]:
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

## 한글을 불러올 땐 strip accents = False 필수

vocab = './data/vocab.txt'
tokenizer = BertWordPieceTokenizer.from_file(vocab=vocab,strip_accents=False)


encoded = tokenizer.encode('python과 javascript를 만들며 고민했다')
print(encoded.tokens)

['[CLS]', 'python', '##과', 'javascript', '##를', '만들며', '고민', '##했다', '[SEP]']


### Transformer로 daistillkobert와 kobert 불러오기

In [88]:
from transformers import DistilBertModel,BertModel, AutoTokenizer

model = DistilBertModel.from_pretrained('monologg/distilkobert')

tokenizer = AutoTokenizer.from_pretrained("monologg/distilkobert")

Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Tokenzier max 개수 세기 

In [78]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("monologg/distilkobert")

sentences = pd.read_csv('./data/bookraw_list.txt')
sentence_list = sentences.iloc[:,0]

# nan 제거
sentence_list = sentence_list[sentence_list.isna() == False].values

# to numpy

### token 상위 99% 이상인 문장 제거 
num_sen = [len(sen.split(' ')) for sen in sentence_list.tolist()]

percentile_99 = np.quantile(num_sen,0.99)

sentence_list_under_99 = sentence_list[np.array(num_sen) < percentile_99 ].tolist()

print(f"최대 토큰 개수 {percentile_99}")


최대 토큰 개수 30.0


In [ ]:
len_max = 0
max_num = 0
for num, sen in enumerate(sentence_list_under_99) :
    
    len_sen = len(tokenizer.encode(sen))

    if len_sen > len_max :
        max_num = num

    len_max = max(len_max,len_sen)

In [75]:
print(f'max_length_minimum: {len_max}')

max_length_minimum: 31


### Fine Tuning 용 Tokenizing

### The tokenizer.encode_plus function combines multiple steps for us:

- Split the sentence into tokens.
- Add the special [CLS] and [SEP] tokens.
- Map the tokens to their IDs.
- Pad or truncate all sentences to the same length.
- Create the attention masks which explicitly differentiate real tokens from [PAD] tokens.


In [80]:
input_ids = []
attention_masks = []

for sent in sentence_list_under_99 : 
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 40,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

In [94]:
sentence_list_under_99[2]

'메타버스 플랫폼의 차이점을 알아보자'

In [98]:
tokenizer.decode([2,0,0,0,0,3])

'[CLS] [UNK] [UNK] [UNK] [UNK] [SEP]'